# Find the dimer
This is a simple example to illustrate how to search for the structures containing C-C or N-N dimers from [Materials Project](http://materialsproject.org)

**Tip**: this is just a toy example to illustrate the functionality. If you want to use it for production. Better run it with pure \*.py script.

**Copyright**: qiang.zhu@unlv.edu
**Date Created**: 2018/08/13

In [1]:
from pymatgen import MPRester

mpr = MPRester('fn8WQGgT9rvZAh6H') # insert your keys here

#define your search criteria
#more info can be found at https://materialsproject.org/docs/api

criteria ={"band_gap": {"$gt":1.0},
          "e_above_hull": {"$lt":0.01},
          #"spacegroup.crystal_system": "hexagonal",
          "elements": {"$all": ["N"]},
          "nelements":{"$lt":3},  
          #"pretty_formula": "BaTiO3",
          }

#choose the properties which you are interested:

properties = ["pretty_formula", 
              "material_id", 
              "spacegroup.symbol", 
              "e_above_hull",
              "structure",
             ]
entries = mpr.query(criteria=criteria, properties=properties)

In [2]:
# output the number of entries which satisfy the above criteria
len(entries)

49

In [3]:
# a simple function to find the C-C dimers
def compute_dimer(struc, specie='C', dist=1.6):

    for site in struc.sites:
        if site.species_string==specie:
            a = struc.get_neighbors(site, r=dist)
            if len(a)==1 and a[0][0].species_string==specie:
                return True
    return False

In [4]:
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import pandas as pd

formula = []
e_above_hull = []
spg = []
mid = []

for entry in entries:
    struc=entry['structure']
    finder = SpacegroupAnalyzer(struc,symprec=0.06,angle_tolerance=5)
    struc = finder.get_conventional_standard_structure()
    if compute_dimer(struc, specie='N'):
        formula.append(entry['pretty_formula'])
        spg.append(entry['spacegroup.symbol'])
        e_above_hull.append(entry['e_above_hull'])
        mid.append(entry['material_id'])
        
col_name = {'Formula':    formula,
            'Space group': spg,
            'e_above_hull': e_above_hull,
            'material_id': mid,
            }
df = pd.DataFrame(col_name)
df = df.sort_values(['e_above_hull', 'Space group'], ascending=[True, False])

In [5]:
df

,Formula,Space group,e_above_hull,material_id
12,BaN6,P2_1/m,0.000000e+00,mp-1707
10,WN2,P-6m2,0.000000e+00,mp-999549
8,CsN3,I4/mcm,0.000000e+00,mp-510557
16,RbN3,I4/mcm,0.000000e+00,mp-743
17,KN3,I4/mcm,0.000000e+00,mp-827
5,CaN6,Fddd,0.000000e+00,mp-676
14,SrN6,Fddd,0.000000e+00,mp-2131
11,NaN3,Cm,0.000000e+00,mp-1064952
13,LiN3,C2/m,0.000000e+00,mp-2659
2,N2,R-3c,8.881784e-16,mp-568584
